In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, cv2 
from multiprocessing import Pool
import threading

We first resize each single image to 512x512 :

https://www.kaggle.com/tchaye59/hpa512x512dataset

https://www.kaggle.com/tchaye59/hpa-512x512-dataset

# This notebook creates a new dataset by combining the four channels in a single file

The directory /kaggle/working is only disk limited to 20GB because everything there is stored forever on when you hit Save > Run All.

[We can have much more disk space outside this directory ](https://www.kaggle.com/product-feedback/155538#872885)(ex. create a directory /kaggle/tmp, you can write many GBs there and it won't count towards your disk usage, but also won't be saved on commit).

In [ ]:
! mkdir -p /root/.kaggle/
! cp ../input/api-token/kaggle.json /root/.kaggle/kaggle.json

In [ ]:
! mkdir -p /kaggle/tmp/hpa_stacked
!kaggle datasets init -p /kaggle/tmp/hpa_stacked

In [ ]:
%%bash
echo "{
  \"title\": \"HPA: Stacked channels 512x512\",
  \"id\": \"tchaye59/HPASTACKEDCHANNELS512x512\",
  \"licenses\": [
    {
      \"name\": \"CC0-1.0\"
    }
  ]
}" > /kaggle/tmp/hpa_stacked/dataset-metadata.json

In [ ]:
DATA_DIR = "../input/hpa-single-cell-image-classification/"
TRAIN_DIR = os.path.join('../input/hpa512x512dataset',"train/")
TEST_DIR = os.path.join('../input/hpa512x512dataset',"test/")
TRAIN_CSV = '../input/hpa512x512dataset/train.csv'
TEST_CSV = os.path.join(DATA_DIR,"sample_submission.csv")

DS_DIR = '/kaggle/tmp/hpa_stacked/'

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

In [ ]:
# Runs a process in a thread
class Worker(threading.Thread):
    
    def __init__(self, process,args,pbar=None):
        super().__init__()
        self.pbar = pbar
        self.process = process
        self.args = args

    def run(self):
        res = self.process(self.args)
        if self.pbar:
            self.pbar.update(1)

In [ ]:
def worker_fn(args):
    id_,src_path,dest_path = args
    red_image = cv2.imread(src_path+id_+"_red.png", cv2.IMREAD_UNCHANGED)
    green_image = cv2.imread(src_path+id_+"_green.png", cv2.IMREAD_UNCHANGED)
    blue_image = cv2.imread(src_path+id_+"_blue.png", cv2.IMREAD_UNCHANGED)
    yellow_image = cv2.imread(src_path+id_+"_yellow.png", cv2.IMREAD_UNCHANGED)
    stacked_images = np.transpose(np.array([red_image, green_image, blue_image, yellow_image]), (1,2,0))
    #stacked_images=stacked_images.astype(np.uint8) 
    
    np.savez_compressed(dest_path+id_+'.npz',image=stacked_images)
    
    #np.save(dest_path+id_+'.npy',stacked_images,allow_pickle=False)

# Build Stacked images

Training

In [ ]:
src_dir = TRAIN_DIR
dest_dir = DS_DIR+'train/'
os.makedirs(dest_dir,exist_ok=True)

In [ ]:
size = len(train_df.ID)
pbar = tqdm(total=size)
n = 5000
for i in range(0,size,n):
    workers = []
    for id_ in train_df.ID[i:min(size,i+n)]:
        worker =  Worker(worker_fn,(id_,src_dir,dest_dir),pbar=pbar)
        worker.start()
        workers.append(worker)
    for worker in workers:
        worker.join()

testing

In [ ]:
src_dir = TEST_DIR
dest_dir = DS_DIR+'test/'
os.makedirs(dest_dir,exist_ok=True)

In [ ]:
size = len(train_df.ID)
pbar = tqdm(total=size)
n = 5000
for i in range(0,size,n):
    workers = []
    for id_ in test_df.ID[i:min(size,i+n)]:
        worker =  Worker(worker_fn,(id_,src_dir,dest_dir),pbar=pbar)
        worker.start()
        workers.append(worker)
    for worker in workers:
        worker.join()

# Let read one image

In [ ]:
def readImage(id_):
  imgPath = f'{DS_DIR}train/{id_}.npz'
  return np.load(imgPath)['image']

In [ ]:
img = readImage(train_df.ID[0])
img.shape

In [ ]:
# Only display the first 3 channels
plt.imshow(img[:,:,:3])
plt.show()

In [ ]:
! kaggle datasets version -p /kaggle/tmp/hpa_stacked -m "Update" --dir-mode tar
#! kaggle datasets create -p /kaggle/tmp/hpa_stacked -u --dir-mode tar

In [ ]:
! rm -rf /root/.kaggle/kaggle.json

Dataset link : https://www.kaggle.com/tchaye59/hpastackedchannels512x512